## UK Road Casualty Analysis with LightGBM

The Road Casualty Statistics datasets was obtained from:
https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-casualty-2023.csv
https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-vehicle-2023.csv
https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-2023.csv

The variables and values (coding) are described here:
https://doc.ukdataservice.ac.uk/doc/5244/mrdoc/pdf/5244userguide.pdf


### Abstract

The Light-GBM classifier was used on UK vehicle collision dataset from 2023 to model the severity of casualty parameter. Decent baseline performance metrics were achieved with True Positive vs False Positive AUC ~ 0.725 and Precision-Recall AUC ~ 0.417.
 

### Data and objectives

The dataset is made public by the UK government.
The data is provided in 3 individual sets:
- casualty dataset
- vehicle dataset
- collision dataset

All three subsets are linked by accident_reference key.
I want to try and see if I can model the severity of injuries sustained by crash victims using features accross the 3 datasets.

### Data Loading

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# loading the three datasets for 2023
casualties = pd.read_csv("dft-road-casualty-statistics-casualty-2023.csv", low_memory = False)
collisions = pd.read_csv("dft-road-casualty-statistics-collision-2023.csv", low_memory = False)
vehicles = pd.read_csv("dft-road-casualty-statistics-vehicle-2023.csv", low_memory = False)

# transferring to working copies
cas = casualties.copy()
acc = collisions.copy()
veh = vehicles.copy()

# we are looking at over a 100k rows and 10s of features
print(cas.shape)
print(acc.shape)
print(veh.shape)

#### Let's do some initial Exploratory Data Analysis
Casualty severity maps as 1, 2, 3 = Fatal, Serious, Slight. We can see that injuries are predominantly Slight and there is also a significant proportion that are classified as Serious.

In [ ]:
# Let's look at casualty severity, speed limit and age of casualty variable distributions to get a feel for it

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
sns.countplot(x='casualty_severity', data=cas)
plt.title('Casualty Severity Distribution')
plt.subplot(1, 3, 2)
sns.histplot(acc['speed_limit'].dropna(), bins=10, kde=False)
plt.title('Speed Limit Distribution')
plt.subplot(1, 3, 3)
sns.histplot(cas['age_of_casualty'].dropna(), bins=10, kde=False)
plt.title('Age of Casualty Distribution')
plt.tight_layout()
plt.show()


## Preprocessing and Feature Engineering 

#### CASUALTY dataset cleaning - let's remove some columns and create new columns if necessary
We'll use:
accident_reference,
vehicle_reference,
casualty_class_lbl,
sex_of_casualty_lbl,
age_band,
target which will be binary - 0 for Slight and 1 for Serious or Fatal

In [ ]:
# Let's map casualty_class so they are easier to understand -----------------------------------------------------
cas_map = {
     1: "driver",
     2: "passenger",
     3: "pedestrian"
}

# creating a casualty class label column
cas["casualty_class_lbl"] = (
    cas["casualty_class"]
          .map(cas_map)
          .fillna("other")
)
# categorical
cas["casualty_class_lbl"] = cas["casualty_class_lbl"].astype("category")


# let's label sex_of_casualty so it's clearer and create a new sex_of_casualty_lbl column ------------------------------------------
sex_map = {
    1: "male",
    2: "female"
}

cas["sex_of_casualty_lbl"] = (
    cas["sex_of_casualty"]
          .map(sex_map)
          .fillna("unknown")   # handles any unexpected code
)

# categorical
cas["sex_of_casualty_lbl"] = cas["sex_of_casualty_lbl"].astype("category")
#cas["age_of_casualty"].value_counts()
#cas["sex_of_casualty_lbl"].value_counts()


# let's put age_of_casualty into age_band bins -------------------------------------------------------------------------------------
age_bins = [0, 17, 30, 50, 70, 99]
age_labels = ["0-17", "18-30", "31-50", "51-70", "71-99"]

cas["age_band"] = pd.cut(
    cas["age_of_casualty"].replace(-1, np.nan),
    bins=age_bins,
    labels=age_labels,
    right=True
).astype(object).fillna("unknown") # taking care on -1 (NaN) by marking them as unknown

# we'll make them categorical in a way where order matters
cas["age_band"] = pd.Categorical(
    cas["age_band"],
    categories=["0-17", "18-30", "31-50", "51-70", "71-99", "unknown"],
    ordered=True
)

# let's get casualty_severity into two binary bins -------------------------------------------------------------------------------------------------

severity_map = {
    1: "Fatal",
    2: "Serious",
    3: "Slight"
}

cas["casualty_severity_lbl"] = (
    cas["casualty_severity"]
        .map(severity_map)
        .fillna("Unknown")  # handling unexpected codes like -1
)

# Making casualty severity ordered
cas["casualty_severity_lbl"] = pd.Categorical(
    cas["casualty_severity_lbl"],
    categories=["Slight", "Serious", "Fatal"],  # ← ascending order
    ordered=True
)

# Create binary target: 1 = Serious or Fatal, 0 = Slight
cas["target"] = cas["casualty_severity_lbl"].isin(["Serious", "Fatal"]).astype(int)

# ok now let's only keep what we need -----------------------------------------------------------------------------------------------
cas_simple = cas[[
    "accident_reference",
    "vehicle_reference",
    "casualty_class_lbl",
    "sex_of_casualty_lbl",
    "age_band",
    "target"
]].copy()

# converting accident reference to category
cas_simple["accident_reference"] = cas_simple["accident_reference"].astype("category")

#cas_simple.info()
#cas_simple.head()



#### COLLISION dataset cleaning - let's remove some columns and create new columns if necessary
We'll use:
accident_reference,
number_of_vehicles,
day_of_week,
road_type_grp,
light_conditions_grp,
road_surface_conditions_grp,
area_type_lbl


In [ ]:
# day_of_week to ordered category -----------------------------------------------------------------------------------------------------

day_map = {
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday"
}

acc["day_of_week_lbl"] = acc["day_of_week"].map(day_map)

acc["day_of_week_lbl"] = pd.Categorical(
    acc["day_of_week_lbl"],
    categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    ordered=True
)


# road_type to category and bins -----------------------------------------------------------------------------------------------------
# Define mapping using a dictionary
road_type_map = {
    1: "roundabout",
    2: "unidirection",
    3: "unidirection",
    4: "unidirection",
    5: "bidirection",
    6: "bidirection",
    7: "bidirection",
    8: "bidirection",
    9: "bidirection"
}

# Apply the mapping
acc["road_type_grp"] = acc["road_type"].map(road_type_map).fillna("unknown")

# Categorical
acc["road_type_grp"] = acc["road_type_grp"].astype("category")

# Light conditions -----------------------------------------------------------------------------------------------------------------------

light_map = {
    1: "daylight",
    2: "daylight",
    3: "daylight",
    4: "darkness",
    5: "darkness",
    6: "darkness",
    7: "darkness"
}

# Apply the mapping
acc["light_conditions_grp"] = acc["light_conditions"].map(light_map).fillna("unknown")

# Categorical
acc["light_conditions_grp"] = acc["light_conditions_grp"].astype("category")


# Road surface conditions -------------------------------------------------------------------------------------------------------------------------

surface_map = {
    1: "dry",
    2: "wet",
    3: "snow",
    4: "frost",
    5: "flood",
    -1: "unknown",
    9: "unknown"
}

# Apply the mapping
acc["road_surface_conditions_grp"] = acc["road_surface_conditions"].map(surface_map).fillna("unknown")

# Convert to ordered categorical
acc["road_surface_conditions_grp"] = pd.Categorical(
    acc["road_surface_conditions_grp"],
    categories=["dry", "wet", "frost", "snow", "flood", "unknown"], # we will drop 'unknown' after merging the datasets
    ordered=True
)

# Urban or rural -----------------------------------------------------------------------------------------------------------------------------------
area_map = {
    1: "urban",
    2: "rural",
    -1: "unknown",
    3: "unknown"
}

acc["area_type_lbl"] = acc["urban_or_rural_area"].map(area_map).fillna("unknown")

# to categorical
acc["area_type_lbl"] = acc["area_type_lbl"].astype("category")


# keeping only what we need ------------------------------------------------------------------------------------------------------------------------
acc_simple = acc[[
    "accident_reference",
    "number_of_vehicles",
    "day_of_week_lbl",
    "road_type_grp",
    "light_conditions_grp",
    "road_surface_conditions_grp",
    "area_type_lbl"
]].copy()

# converting accident reference to category
acc_simple["accident_reference"] = acc_simple["accident_reference"].astype("category")

#acc_simple.info()
#acc_simple.head()
#acc["urban_or_rural_area"].unique()
#acc["urban_or_rural_area"].value_counts()


#### VEHICLE dataset cleaning - let's remove some columns and create new columns if necessary
We'll use:
accident_reference,
vehicle_reference,
vehicle_type_grp,
sex_of_driver_lbl,
age_band_driver,
skid_overturn_grp

In [ ]:

# Combining some vehicle types for simplicity---------------------------------------------------------------------
veh_map = {
     1 : "bicycle",
     2 : "motorcycle",  3 : "motorcycle",  4 : "motorcycle",  5 : "motorcycle",
     8 : "car",         9 : "car",
    10 : "bus",        11 : "bus",
    17 : "agricultural",
    18 : "tram",
    19 : "goods",      20 : "goods",      21 : "goods"
}

# Creating vehicle type groups
veh["vehicle_type_grp"] = veh["vehicle_type"].map(veh_map).fillna("other")

veh["vehicle_type_grp"] = veh["vehicle_type_grp"].astype("category")

veh["vehicle_type_grp"].value_counts()
#print(veh["vehicle_type_grp"].cat.categories)


# sex of driver ------------------------------------------------------------------------------------------------------------------

sex_map = {
    1: "male",
    2: "female",
    3: "unknown"
}

veh["sex_of_driver_lbl"] = veh["sex_of_driver"].map(sex_map).fillna("unknown")
veh["sex_of_driver_lbl"] = veh["sex_of_driver_lbl"].astype("category")


# Age of driver ------------------------------------------------------------------------------------------------------------------------
# let's identify -1s first
veh["age_of_driver_clean"] = veh["age_of_driver"].replace(-1, np.nan)

# now let's bin them
age_bins = [0, 17, 30, 50, 70, float('inf')]
age_labels = ["0-17", "18-30", "31-50", "51-70", "70+"]

veh["age_band_driver"] = pd.cut(
    veh["age_of_driver_clean"],
    bins=age_bins,
    labels=age_labels,
    right=True
).astype(object).fillna("unknown")

# let's make it categorical and ordered just in case
veh["age_band_driver"] = pd.Categorical(
    veh["age_band_driver"],
    categories=["0-17", "18-30", "31-50", "51-70", "70+", "unknown"],
    ordered=True
)


# Skidding and overturning ---------------------------------------------------------------------------------------------------------------------
# if skidded and not overturned then class as skidded, if skidded and overturned then class as overturned

s_o_map = {
    0: "no",
    1: "skidded",
    2: "overturned",
    3: "skidded",
    4: "overturned",
    5: "overturned",
    -1: "unknown",
    9: "unknown"
}

veh["skid_overturn_grp"] = veh["skidding_and_overturning"].map(s_o_map).fillna("unknown")

veh["skid_overturn_grp"] = veh["skid_overturn_grp"].astype("category")

#veh["skidding_and_overturning"].unique()
#veh["skidding_and_overturning"].value_counts()
#veh["skid_overturn_grp"].value_counts()

# keeping only what we need
veh_simple = veh[[
    "accident_reference",
    "vehicle_reference",
    "vehicle_type_grp",
    "sex_of_driver_lbl",
    "age_band_driver",
    "skid_overturn_grp"
]].copy()

# converting accident reference to category
veh_simple["accident_reference"] = veh_simple["accident_reference"].astype("category")

#veh_simple.info()
#veh_simple.head()

### Merging the 3 datasets

In [ ]:
# first casualties and vehicles
cas_veh = pd.merge(
    cas_simple,
    veh_simple,
    on=["accident_reference", "vehicle_reference"],
    how="left",
    validate="many_to_one"  # so that each casualty links to one vehicle
)

# and now cas_veh with the collision dataset
full_df = pd.merge(
    cas_veh,
    acc_simple,
    on="accident_reference",
    how="left",
    validate="many_to_one"  # so that each casualty links to one accident
)

# let's drop rows with unknown/missing surface conditions
full_df = full_df[full_df["road_surface_conditions_grp"] != "unknown"]

# we will also drop unknowns from urban or rural area type
full_df = full_df[full_df["area_type_lbl"] != "unknown"]


#print(full_df.shape)
#full_df.isna().sum()

### More EDA. "target" is 0 for Slight casualty, 1 for Serious or Fatal casualty

In [ ]:
eda_df = full_df.sample(20000, random_state=5)

plt.figure(figsize=(16, 10))

plt.subplot(2, 2, 1)
sns.countplot(x='target', hue='casualty_class_lbl', data=eda_df)
plt.title('Target by Casualty Class')

plt.subplot(2, 2, 2)
sns.countplot(x='target', hue='vehicle_type_grp', data=eda_df)
plt.title('Target by Vehicle Type')

plt.subplot(2, 2, 3)
sns.countplot(x='target', hue='sex_of_casualty_lbl', data=eda_df)
plt.title('Target by Sex of Casualty')

plt.subplot(2, 2, 4)
sns.countplot(x='target', hue='area_type_lbl', data=eda_df)
plt.title('Target by Urban/Rural Area')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 5))

plt.subplot(1, 2, 1)
sns.boxplot(x='target', y='number_of_vehicles', data=eda_df)
plt.title('Number of Vehicles by Target')

plt.subplot(1, 2, 2)
sns.countplot(x='age_band_driver', hue='target', data=eda_df)
plt.title('Driver Age Band by Target')

plt.tight_layout()
plt.show()


## Modelling 

#### Ok, now let's split the data and start modelling using a Gradient Boosted Decision Tree (GBDT) classifier as it deals relatively well with imbalanced datasets.

In [ ]:
# dropping what we don't need as predictors
X  = full_df.drop(columns=["target", "accident_reference", "vehicle_reference"])
y = full_df["target"]

# splitting into 80/20 train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,     # preserving class balance
    random_state=5 # reproducibility
)

#print("Train shape:", X_train.shape)
#print("Test shape:", X_test.shape)
#print("Target distribution in train:")
#print(y_train.value_counts(normalize=True))
#print("Target distribution in test:")
#print(y_test.value_counts(normalize=True))

# splits look very similarly balanced so should be good to go

In [ ]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from lightgbm import LGBMClassifier

# take categorical columns
categorical_cols = X.select_dtypes(include="category").columns.tolist()
#print("Categorical features:", categorical_cols)


clf = LGBMClassifier(
    objective="binary",
    class_weight="balanced", # this addresses the class imbalance (0.21/0.79) in the target variable
    learning_rate=0.1, # good default
    num_leaves=30, # I tried different # leaves, 30 seems to work ok
    n_estimators=1000,
    random_state=5
)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    eval_metric="auc",
    categorical_feature=categorical_cols,
    callbacks=[
        early_stopping(50),
        log_evaluation(50)
    ]
)


## Evaluation

#### Evaluating performance - trying different threshold values

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[:, 1]

# trying out thresholds 0.3, 0.4 and 0.5 to try and find the optimum value
for thresh in [0.3, 0.4, 0.5]:
    y_pred_thresh = (y_pred_proba >= thresh).astype(int)
    print(f"\nThreshold: {thresh}")
    print(confusion_matrix(y_test, y_pred_thresh))
    print(classification_report(y_test, y_pred_thresh))


#print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
#print("\nClassification Report:")
#print(classification_report(y_test, y_pred))
#print("\nConfusion Matrix:")
#print(confusion_matrix(y_test, y_pred))

#### Evaluating performance - threshold value of 0.5 seems to be provide a balanced tradeoff between precision and recall. 
Only 34% of all predicted serious/fatal cases were actually serious/fatal so it's over-predicting a bit. The model was able to find 70% of all actual serious/fatal cases. Both Precision and Recall for Class 1 are better than random chance (0.34 vs 0.21 and 0.70 vs 0.5 respectively)

#### Evaluating performance - visualising Precision-Recall tradeoff as a function on Threshold

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.title("Precision-Recall vs Threshold")
plt.legend()
plt.grid(True)
plt.show()

#### Evaluating performance 
- visualising ROC curve and Precision-Recall Curve. The model achieved ROC AUC of 0.725 which is good, but not great. Precision-Recall AUC was 0.417 which is not great but it is still almost double of what it would be by pure chance.

In [ ]:
# Predicted probabilities for the positive class
y_pred_proba = clf.predict_proba(X_test)[:, 1]

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)

# Plot both curves
plt.figure(figsize=(14, 6))

# ROC Curve
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("ROC Curve")
plt.legend()

# Precision-Recall Curve
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f"AUC = {pr_auc:.3f}", color="darkorange")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()

plt.tight_layout()
plt.show()

#### Determining feature importance 
- suprisingly, the number of vehicles feature was used over 300 times to split the data so it was "the most contributing" feature.
- Also surprisingly - the road surface conditions grouping had the lowest contribution, although this may be due to it being correlated with another feature.

In [ ]:
# Get feature importances
feature_imp = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.title("Feature Importance (LightGBM)")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

## Conclusion
In conclusion, using a LGBM Classifier resulted in solid baseline.
True Positive vs False Positive AUC ~ 0.725 → The model ranks cases well, above chance, but not strongly predictive.

Precision-Recall AUC ~ 0.417 → Better than random (~0.21), but still indicates lots of false positives when trying to detect serious/fatal outcomes.

## Potential Improvements
Aspects that could be considered to improve the model:
- Additional feature engineering could help - maybe some traffic and/or environmental contex.
- Combined features - e.g. - driver age and casualty class.
- Maybe trying different parameter values (threshold, # leaves, estimators)
- Trying a different model - e.g. - Random Forest, CatBoost.

### Key Package Information

In [ ]:
import pkg_resources

for dist in pkg_resources.working_set:
    if dist.project_name.lower() in [
        "pandas", "numpy", "matplotlib", "seaborn", "lightgbm", "scikit-learn"
    ]:
        print(f"{dist.project_name}=={dist.version}")